### Load Package and Data

In [22]:
import pandas as pd
import joblib
import pandas as pd
import numpy as np
from typing import Tuple, Sequence, List, Optional, Iterable, Any, Iterator, Callable
import textdistance

def levenshtein_similarity(truth: List[str], pred: List[str]) -> float:
    assert len(truth) == len(pred)
    scores = sum(textdistance.levenshtein.normalized_similarity(t, p) for t, p in zip(truth, pred))
    return scores / len(truth)

def full_sentence_accuracy(truth: List[str], pred: List[str]) -> float:
    """Calculate the number of exact matches."""
    assert len(truth) == len(pred)
    correct_count = sum(int(t == p) for t, p in zip(truth, pred))
    # for t,p in zip(truth,pred):
    #     if t != p:
    #         print(t,p)
    return correct_count / len(truth)

#truth_file = '../data/test_300.csv'
truth_file = '../data/test_1022.csv'

predict_file = 'results/predictions/prediction_1022_of_saved_models-merged_models-saved_models-llama2_13b_chat_qlora-split_train_800_lora_r64_lr0.0001-checkpoint-800.csv'
truth_df = pd.read_csv(truth_file, encoding='utf-8').fillna('N/A')
predict_df = pd.read_csv(predict_file, encoding='utf-8').fillna('N/A')
predict_df

,Generated Text,Actual Text,Paragraph,Generated Text Prob,IUPAC,1H NMR text,1H NMR conditions,1H NMR data,13C NMR text,13C NMR conditions,...,DSC text,DSC conditions,DSC data,TGA text,TGA conditions,TGA data,29Si NMR text,29Si NMR conditions,29Si NMR data,nan
0,"{'IUPAC': 'nemorosone', '1H NMR text': '1H NMR...","{""IUPAC"":""nemorosone"",""1H NMR text"":""1H NMR (C...","Purity of isolated nemorosone was >99%, as det...",0.387806,nemorosone,"1H NMR (CD3OD, 400 MHz): 7.53 (br d, J = 7.7 H...","CD3OD, 400 MHz","7.53 (br d, J = 7.7 Hz, 2H), 7.43 (app tt, J =...","13C NMR (CD3OD, 100 MHz): 209.26 (C), 194.88 (...","CD3OD, 100 MHz",...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,"{'IUPAC': 'Mach', '1H NMR text': '1H-NMR (500 ...","{""IUPAC"":""Mach"",""1H NMR text"":""1H-NMR (500 MHz...",The method used to isolate Mach from S. chinen...,0.805737,Mach,"1H-NMR (500 MHz, CDCl3) assign data: δ 6.92~6....","500 MHz, CDCl3","6.92~6.82 (6H, m, aromatic protons), 6.33 (1H,...","13C-NMR (125 MHz, CDCl3) assign data: δ 151.0 ...","125 MHz, CDCl3",...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,{'IUPAC': 'N-(3-chlorophenyl)-N-((5-(hydrazine...,"{""IUPAC"":""N-(3-chlorophenyl)-N-((5-(hydrazinec...","Compound 3 (0.35 g, 1.00 mmol, 1.0 equiv.) was...",0.875487,N-(3-chlorophenyl)-N-((5-(hydrazinecarbonyl)py...,"1H NMR (401 MHz, CD3OD): δ 8.92 (dd, J = 0.8, ...","401 MHz, CD3OD","8.92 (dd, J = 0.8, 2.2 Hz, 1H), 8.22 (dd, J = ...","13C NMR (101 MHz, CD3OD): δ 166.6, 162.3, 149....","101 MHz, CD3OD",...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,{'IUPAC': 'N-(3-chlorophenyl)-N-((5-(5-(difluo...,"{""IUPAC"":""N-(3-chlorophenyl)-N-((5-(5-(difluor...","Compound 5 (24 mg, 0.05 mmol, 1.0 equiv.) and ...",0.886407,N-(3-chlorophenyl)-N-((5-(5-(difluoromethyl)-1...,"1H NMR (401 MHz, CDCl3): δ 9.24 (d, J = 1.6 Hz...","401 MHz, CDCl3","9.24 (d, J = 1.6 Hz, 1H), 8.37 (dd, J = 2.2, 8...","13C NMR (101 MHz, CDCl3): δ 164.1, 160.9, 158....","101 MHz, CDCl3",...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,"{'IUPAC': 'pulchin A', '1H NMR text': 'The 1H ...","{""IUPAC"":""Pulchin A"",""1H NMR text"":""The 1H NMR...",Pulchin A (1) was obtained in the form of whit...,0.004595,pulchin A,The 1H NMR spectral data exhibited four tertia...,N/A,"0.91, 0.94, 1.05, 1.18, 3.48, 3.74, 4.71",The 13C NMR spectral data exhibited seven carb...,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,"{'IUPAC': '3,3′-Diisopropyl-[1,1′-bi\xadnaphth...","{""IUPAC"":""bis-S-thio­carbamate ester"",""1H NMR ...","3,3′-Diisopropyl-[1,1′-bi­naphthalene]-2,2′-di...",0.321980,"3,3′-Diisopropyl-[1,1′-bi­naphthalene]-2,2′-di...",1H NMR (200 MHz) δ (ppm): 1.34 and 1.46 [2 × (...,200 MHz,"1.34, 1.46, 2.51, 3.66, 7.04, 7.11, 7.39, 7.84...",13C NMR (50 MHz) δ (ppm): 23.4 and 24.8 (2 × C...,50 MHz,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1018,"{'IUPAC': '(2,6-Diiso\xadpropyldi\xadnaphtho\x...","{""IUPAC"":""(2,6-Diiso­propyldi­naphtho­[2,1-d:1...","(2,6-Diiso­propyldi­naphtho­[2,1-d:1′,2′-f][1,...",0.546119,"(2,6-Diiso­propyldi­naphtho­[2,1-d:1′,2′-f][1,...","1H NMR (200 MHz) δ (ppm): 1.26, 1.34, 1.35 and...",200 MHz,"1.26, 1.34, 1.35, 1.61, 3.26, 3.61, 3.97, 4.27...","13C NMR (50 MHz) δ (ppm): 22.9, 24.0, 24.2 and...",50 MHz,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1019,"{'IUPAC': 'Isopropyl 2,6,6-trimethyl-4-(3-fluo...","{""IUPAC"":""Isopropyl 2,6,6-trimethyl-4-(3-fluor...","Isopropyl 2,6,6-trimethyl-4-(3-fluoro-5-tri­fl...",0.702957,"Isopropyl 2,6,6-trimethyl-4-(3-fluoro-5-tri­fl...","1H NMR (400 MHz, DMSO-d 6): δ 0.82 (3H, s, 6-C...","400 MHz, DMSO-d 6","0.82 (3H, s, 6-CH3), 0.91 [3H, d, J = 6.4 Hz, ...","13C NMR (100 MHz, DMSO-d 6): δ 18.2 (2-CH3), 2...","100 MHz, DMSO-d 6",...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1020,"{'IUPAC': '(2)', '1H NMR text': '1H NMR (400\u...","{""IUPAC"":""2"",""1H NMR text"":""1H NMR (400 MHz, C...",The synthesis of (2) is shown schematically in...,0.316626,(2),"1H NMR (400 MHz, C

### Evaluate

In [ ]:
ground_truth_IUPAC = []
ground_truth_1H_NMR = []
ground_truth_1H_NMR_solvent = []
ground_truth_1H_NMR_data = []
ground_truth_13C_NMR = []
ground_truth_13C_NMR_solvent = []
ground_truth_13C_NMR_data = []

predict_IUPAC = []
predict_1HNMR = []
predict_1HNMR_solvent = []
predict_1HNMR_data = []
predict_13CNMR = []
predict_13CNMR_solvent = []
predict_13CNMR_data = []

# ground truth
for i,row in truth_df.iterrows():
    ground_truth_13C_NMR.append(row['13C NMR text'])
    ground_truth_IUPAC.append(row['IUPAC'])
    ground_truth_1H_NMR.append(row['1H NMR text'])
    ground_truth_1H_NMR_solvent.append(row['1H NMR conditions'])
    ground_truth_1H_NMR_data.append(row['1H NMR data'])
    ground_truth_13C_NMR.append(row['13C NMR text'])
    ground_truth_13C_NMR_solvent.append(row['13C NMR conditions'])
    ground_truth_13C_NMR_data.append(row['13C NMR data'])

# prediction
for i,row in predict_df.iterrows():
    predict_13CNMR.append(row['13C NMR text'])
    predict_IUPAC.append(row['IUPAC'])
    predict_1HNMR.append(row['1H NMR text'])
    predict_1HNMR_solvent.append(row['1H NMR conditions'])
    predict_1HNMR_data.append(row['1H NMR data'])
    predict_13CNMR.append(row['13C NMR text'])
    predict_13CNMR_solvent.append(row['13C NMR conditions'])
    predict_13CNMR_data.append(row['13C NMR data'])

# Evaluation
print("levenshtein_similarity_of_IUPAC: ", levenshtein_similarity(ground_truth_IUPAC, predict_IUPAC))
print("levenshtein_similarity_of_1HNMR_text: ", levenshtein_similarity(ground_truth_1H_NMR, predict_1HNMR))
print("levenshtein_similarity_of_1HNMR_conditions: ", levenshtein_similarity(ground_truth_1H_NMR_solvent, predict_1HNMR_solvent))
print("levenshtein_similarity_of_1HNMR_data: ", levenshtein_similarity(ground_truth_1H_NMR_data, predict_1HNMR_data))
print("levenshtein_similarity_of_13CNMR_text: ", levenshtein_similarity(ground_truth_13C_NMR, predict_13CNMR))
print("levenshtein_similarity_of_13CNMR_conditions: ", levenshtein_similarity(ground_truth_13C_NMR_solvent, predict_13CNMR_solvent))
print("levenshtein_similarity_of_13CNMR_data: ", levenshtein_similarity(ground_truth_13C_NMR_data, predict_13CNMR_data))
print()
print("full_sentence_accuracy_of_IUPAC: ", full_sentence_accuracy(ground_truth_IUPAC, predict_IUPAC))
print("full_sentence_accuracy_of_1HNMR_text: ", full_sentence_accuracy(ground_truth_1H_NMR, predict_1HNMR))
print("full_sentence_accuracy_of_1HNMR_conditions: ", full_sentence_accuracy(ground_truth_1H_NMR_solvent, predict_1HNMR_solvent))
print("full_sentence_accuracy_of_1HNMR_data: ", full_sentence_accuracy(ground_truth_1H_NMR_data, predict_1HNMR_data))
print("full_sentence_accuracy_of_13CNMR_text: ", full_sentence_accuracy(ground_truth_13C_NMR, predict_13CNMR))
print("full_sentence_accuracy_of_13CNMR_conditions: ", full_sentence_accuracy(ground_truth_13C_NMR_solvent, predict_13CNMR_solvent))
print("full_sentence_accuracy_of_13CNMR_data: ", full_sentence_accuracy(ground_truth_13C_NMR_data, predict_13CNMR_data))

###################################################################################################
columns_to_compare = ['IUPAC','1H NMR data']

# Iterate through each row and evaluate
correct_count = 0
total_rows = len(predict_df)

for index, row in predict_df.iterrows():
    ground_truth_row = truth_df.loc[index, columns_to_compare]
    prediction_row = row[columns_to_compare]
    
    # Check if all specified columns in the current row are equal to the ground truth
    if prediction_row.equals(ground_truth_row):
        correct_count += 1

# Calculate accuracy based on specified columns
accuracy = correct_count / total_rows


print("The accuracy of IUPAC and 1H_NMR_data being correct at the same time: ", accuracy)

######################################################################################
columns_to_compare = ['IUPAC','1H NMR data']
# Iterate through each row and evaluate
correct_count = 0
total_rows = len(predict_df)

for index, row in predict_df.iterrows():
    ground_truth_row = truth_df.loc[index, columns_to_compare]
    prediction_row = row[columns_to_compare]
    
    # Check if all specified columns in the current row are equal to the ground truth
    if prediction_row.equals(ground_truth_row):
        correct_count += 1

# Calculate accuracy based on specified columns
accuracy = correct_count / total_rows


print("The accuracy of IUPAC and 13C_NMR_data being correct at the same time: ", accuracy)
